In [1]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('.') if f.endswith('.json')]

In [2]:
onlyfiles

['dummy_total_time_18393.58_max_mem_allocated_26158072832_max_mem_reserved_37052481536.json']

In [7]:
import re
import json
import pandas as pd

def compute_metric(output_filename):
    with open(output_filename, 'r') as f:
        run_results = json.load(f)
    total_acc = 0
    total_num = 0
    for task in run_results:
        acc = 0
        pred_answers = run_results[task]['pred_answers']
        gold_answers = run_results[task]['gold_answers']
        for pred, gold in zip(pred_answers, gold_answers):
            if pred == gold: acc += 1
        #print("ACC-%s: %.4f" % (task, acc/len(gold_answers)))
        total_acc += acc
        total_num += len(gold_answers)
    #print("ACC-all: %.4f" % (total_acc/total_num))
    return total_acc/total_num
    
def extract_stats(file):
    #acceleration = re.search('run_results_(.*?)_',file).group(1)
    #model_type = re.search('(?s:.*)_(.*?)_total_time',file).group(1)
    time = re.search('_total_time_(.*?)_max_mem_allocated_',file).group(1)
    max_mem_allocated = re.search('_max_mem_allocated_(.*?)_max_mem_reserved_',file).group(1)
    max_mem_reserved = re.search('_max_mem_reserved_(.*?).json',file).group(1)
    
    res = pd.DataFrame([
    {
        #"acceleration": acceleration,
        #"model_type": model_type,
        "time": float(time),
        "max_mem_allocated": int(max_mem_allocated)/1000000,
        "max_mem_reserved": int(max_mem_reserved)/1000000,
        "accuracy": compute_metric(file)
    }]
    )
    return res
    # print(compute_metric(file))
    # print(acceleration)
    # print(model_type)
    # print(float(time))
    # print(int(max_mem_allocated)/1000000)
    # print(int(max_mem_reserved)/1000000)

In [8]:

final_results = pd.DataFrame(
    [{
        #"acceleration": [],
        #"model_type": [],
        "time": [],
        "max_mem_allocated": [],
        "max_mem_reserved": [],
        "accuracy": []
    }]
)

for i in onlyfiles:
    final_results = pd.concat([extract_stats(i), final_results], ignore_index=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
print('Results RUMMLU')
final_results.iloc[:-1]

Results RUMMLU


,acceleration,model_type,time,max_mem_allocated,max_mem_reserved,accuracy
0,gguf,mistral7-4bitb,834.42,6616.252416,6616.252416,0.457119
1,gguf,mistral7-8bitb,763.39,10137.370624,10137.370624,0.464403
2,flash2,mistral7b,1523.75,19863.1168,39808.139264,0.463582
3,default,mistral7b,2011.1,24508.253696,39267.074048,0.464095
4,deepspeed,mistral7b,2009.48,24508.253696,39267.074048,0.464095
5,gguf,llama7-4bitb,757.19,7914.389504,7914.389504,0.314936
6,gguf,llama7-8bitb,688.57,11183.849472,11183.849472,0.325503
7,default,llama7b,1822.87,26850.440704,40653.29152,0.326836
8,flash2,llama7b,1380.44,23278.340096,39764.099072,0.326734
9,deepspeed,llama7b,1372.64,27519.563776,28422.701056,0.033238
